## Import data

In [7]:
from brightway2 import *

In [8]:
projects.set_current('Import Quantis OFP - version 4') 

In [0]:
bw2setup()

Biosphere database already present!!! No setup is needed


### Load Quantis data

Das import file wurde wie folgt verändert
- Renaming der Systemprozesse: replace /UK S durch SP/UK
- Renaming der Unitprozesse: replace /UK U durch UP/UK
--> so werden location richtig erkannt (war vorher nicht der Fall)

In [3]:
# sp2 = SimaProCSVImporter('/Users/ioconnor/Google Drive/Eaternity/Science (SCI)/08. LCA/Quantis/EaternityMeatDatasets_v3_iocN_3.CSV', 'MeatDatasets')
sp2 = SimaProCSVImporter('/d/documents/eaternity/data/EaternityMeatDatasets_v3_iocN_3.csv', 'MeatDatasets')
sp2.statistics()

Extracted 294 unallocated datasets in 8.27 seconds
294 datasets
100893 exchanges
100893 unlinked exchanges
  Type biosphere: 2923 unique unlinked exchanges
  Type production: 403 unique unlinked exchanges
  Type technosphere: 621 unique unlinked exchanges


(294, 100893, 100893)

In [5]:
sp2.migrate('simapro-ecoinvent-3.3')
sp2.migrate('default-units')
sp2.apply_strategies()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: normalize_units
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
Applying stra

### Load ecoinvent

In [6]:
# ei2 = SingleOutputEcospold2Importer('/Users/ioconnor/Documents/ecoinvent 3.3/datasets', 'ecoinvent 3.3 cutoff')
ei2 = SingleOutputEcospold2Importer('/d/documents/eaternity/data/3.3/datasets', 'ecoinvent 3.3 cutoff')

Extracting XML data from 13831 datasets
Extracted 13831 datasets in 76.09 seconds


In [7]:
ei2.apply_strategies()
ei2.statistics()

Applying strategy: normalize_units
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: nuncertainty
Applied 13 strategies in 5.88 seconds
13831 datasets
501148 exchanges
0 unlinked exchanges
  


(13831, 501148, 0)

In [8]:
ei2.write_database()

Title: Writing activities to SQLite3 database:
  Started: 03/16/2017 16:04:03
  Finished: 03/16/2017 16:06:36
  Total time elapsed: 00:02:33
  CPU %: 83.40
  Memory %: 15.92
Created database: ecoinvent 3.3 cutoff
Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:02:33


Brightway2 SQLiteBackend: ecoinvent 3.3 cutoff

In [9]:
ei2 = None
# why not: del ei2?

In [10]:
sp2.match_database('ecoinvent 3.3 cutoff', ignore_categories=True)

Applying strategy: link_iterable_by_fields
Couldn't apply strategy link_iterable_by_fields:
	Object in source database can't be uniquely linked to target database.
Problematic dataset is:
{'filename': '(missing)',
 'location': 'RER',
 'name': 'diammonium phosphate production',
 'unit': 'kilogram'}
Possible targets include (at least one not shown):
[{'filename': 'ecd7c263-eab8-4217-8b5c-95c11b094bc3_5321a299-0c03-4bfc-9dda-d020456f6b2e.spold',
  'location': 'RER',
  'name': 'diammonium phosphate production',
  'unit': 'kilogram'}]


In [11]:
db = Database('ecoinvent 3.3 cutoff')

In [12]:
import functools
from bw2io.strategies import link_iterable_by_fields

sp2.apply_strategy(functools.partial(
        link_iterable_by_fields, 
        other=Database('ecoinvent 3.3 cutoff'),
        kind='technosphere',
        fields=['reference product', 'name', 'unit', 'location']
))
sp2.statistics()

Applying strategy: link_iterable_by_fields
403 datasets
101204 exchanges
2508 unlinked exchanges
  Type biosphere: 192 unique unlinked exchanges
  Type technosphere: 26 unique unlinked exchanges


(403, 101204, 2508)

In [13]:
for i, e in enumerate(sp2.unlinked):
    if e['type'] == 'technosphere':
        print(e['name'], e['unit'], e['categories'])
    if i > 20000:
        break

Sugar beet pulp, dried, from pulp drying, at plant Economic/DE kilogram ('Materials/fuels',)
Maize gluten meal, consumption mix, at feed compound plant Economic/NL kilogram ('Materials/fuels',)
Wheat grain, conventional, national average, animal feed, at farm gate kilogram ('Materials/fuels',)
Municipal waste collection service by 21 metric ton lorry {GLO}| market for | Alloc Rec, U ton kilometer ('Electricity/heat',)
Sunflower seed meal, consumption mix, at feed compound plant Economic/NL kilogram ('Materials/fuels',)
Linseed, at farm Economic/DE kilogram ('Materials/fuels',)
Triticale, at farm Economic/DE kilogram ('Materials/fuels',)
Waste in incineration kilogram ('Final waste flows',)
Waste in inert landfill kilogram ('Final waste flows',)
Cathode loss kilogram ('Final waste flows',)
Dross kilogram ('Final waste flows',)
Dross for recycling kilogram ('Final waste flows',)
Aluminium waste kilogram ('Final waste flows',)
Cathode iron ingots waste kilogram ('Final waste flows',)
Mine

### Load both agribalyse  data sets

In [14]:
# sp = SimaProCSVImporter('/Users/ioconnor/Google Drive/Eaternity/Science (SCI)/08. LCA/Quantis/Harvester&BritishGrain.CSV', 'AGs')
sp = SimaProCSVImporter('/d/documents/eaternity/data/Harvester&BritishGrain.CSV', 'AGs')
sp2.statistics()

Extracted 2 unallocated datasets in 0.36 seconds
403 datasets
101204 exchanges
2508 unlinked exchanges
  Type biosphere: 192 unique unlinked exchanges
  Type technosphere: 26 unique unlinked exchanges


(403, 101204, 2508)

In [15]:
sp.migrate('simapro-ecoinvent-3.3')
sp.migrate('default-units')
sp.apply_strategies()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: normalize_units
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
Applying stra

In [16]:
sp.match_database('ecoinvent 3.3 cutoff', ignore_categories=True)

Applying strategy: link_iterable_by_fields


In [17]:
import functools
from bw2io.strategies import link_iterable_by_fields

sp.apply_strategy(functools.partial(
        link_iterable_by_fields, 
        other=Database('ecoinvent 3.3 cutoff'),
        kind='technosphere',
        fields=['reference product', 'name', 'unit', 'location']
))
sp.statistics()

Applying strategy: link_iterable_by_fields
2 datasets
4335 exchanges
107 unlinked exchanges
  Type biosphere: 57 unique unlinked exchanges


(2, 4335, 107)

In [18]:
sp.write_excel()

Wrote matching file to:
/home/oney/.local/share/Brightway3/Import-Quantis-OFP-version-4.ba90a3459a565ec10e14da8d23fa4b2d/output/db-matching-AGs.xlsx


fix necessary biosphere flows

### fix necessary biosphere flows
#### link methane, CO2 (2x),  NO

Meiste ersetzt analog zu Chris script  http://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/IO%20-%20importing%20FORWAST.ipynb.

Beim 2ten CO2 bin ich mir nicht sicher ob fossil oder non-fossil gemeint ist. 
--> ich glaube, non-fossil CO2 hat CF=0? falls ja:
-> AGrigalyse UP nachschauen, welches CO2 gemeint ist...


In [19]:
for i, e in enumerate(sp.unlinked):
    if e['type'] == 'biosphere':
        print(e['name'], e['unit'], e['categories'])
    if i > 20000:
        break

Copper, Cu 0.2%, in mixed ore kilogram ('natural resource', 'in ground')
Gold, Au 1.8E-4%, in mixed ore kilogram ('natural resource', 'in ground')
Gold, Au 5.4E-4%, Ag 1.5E-5%, in ore kilogram ('natural resource', 'in ground')
Gold, Au 6.8E-4%, Ag 1.5E-4%, in ore kilogram ('natural resource', 'in ground')
Gold, Au 9.7E-5%, Ag 7.6E-5%, in ore kilogram ('natural resource', 'in ground')
Lead, Pb 3.6E-1%, in mixed ore kilogram ('natural resource', 'in ground')
Silver, Ag 1.5E-4%, Au 6.8E-4%, in ore kilogram ('natural resource', 'in ground')
Silver, Ag 1.5E-5%, Au 5.4E-4%, in ore kilogram ('natural resource', 'in ground')
Silver, Ag 5.4E-3%, in mixed ore kilogram ('natural resource', 'in ground')
Silver, Ag 7.6E-5%, Au 9.7E-5%, in ore kilogram ('natural resource', 'in ground')
Strontium kilogram ('natural resource', 'in ground')
Transformation, from unspecified, natural square meter ('natural resource', 'land')
Transformation, to annual crop, fallow square meter ('natural resource', 'land')

In [20]:
bp=Database('biosphere3')

In [21]:
bp.search('carbon dioxide ')

['Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, from soil or biomass stock' (kilogram, None, ('air', 'indoor')),

In [22]:
migration_data_bioM = {
    'fields': ['name','categories','type','unit'],
    'data': [
        (
            # First element is input data in the order of `fields` above
            ('Methane',('air',), 'biosphere', 'kilogram'),
            # Second element is new values 
            { 
                'name': 'Methane, fossil',              
            }
        ),
        (
            
            ('Carbon dioxide, in air',('natural resource',), 'biosphere', 'kilogram'),
            
            { 
                'categories': ('natural resource', 'in air',),              
            }
        ),
        (
            # Achtung: nicht sicher ob mit fossil oder non-fossil ersetzen->check CF!!
            ('Carbon dioxide',('air',), 'biosphere', 'kilogram'),
            
            { 
                'name': 'Carbon dioxide, fossil',               
            }
        ),
        (
            
            ('Nitrogen monoxide',('air', 'non-urban air or from high stacks'), 'biosphere', 'kilogram'),
            
            { 
                'name': 'Nitrogen oxides',              
            }
        ) 
    ]
}

Migration('new-ecoinvent-bioM').write(
    migration_data_bioM, 
    description='New datasets in ecoinvent 3.3'
)

In [23]:
sp.migrate('new-ecoinvent-bioM')
sp.apply_strategies()
sp.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: normalize_units
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
Applying strategy: link_iterable_by_fields
Applied 19 strategies in 0.76 seconds
2 dat

(2, 4335, 103)

In [24]:
sp.write_excel()

Wrote matching file to:
/home/oney/.local/share/Brightway3/Import-Quantis-OFP-version-4.ba90a3459a565ec10e14da8d23fa4b2d/output/db-matching-AGs.xlsx


the remaining unlinked biopshere exchanges can be dropped for the moment (only interested in GWP)

In [25]:
sp.drop_unlinked(i_am_reckless=True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.01 seconds


In [26]:
sp.match_database('ecoinvent 3.3 cutoff', ignore_categories=True)

Applying strategy: link_iterable_by_fields


In [27]:
sp.statistics()

2 datasets
4232 exchanges
0 unlinked exchanges
  


(2, 4232, 0)

In [28]:
ag=sp.write_database()

Title: Writing activities to SQLite3 database:
  Started: 03/16/2017 16:09:38
  Finished: 03/16/2017 16:09:40
  Total time elapsed: 00:00:01
  CPU %: 65.20
  Memory %: 14.67
Created database: AGs
Writing activities to SQLite3 database:
0%  100%
[##] | ETA: 00:00:00
Total time elapsed: 00:00:01


In [29]:
ag

Brightway2 SQLiteBackend: AGs

## Solve unlinked exchanges

In [30]:
sp2.statistics()

403 datasets
101204 exchanges
2508 unlinked exchanges
  Type biosphere: 192 unique unlinked exchanges
  Type technosphere: 26 unique unlinked exchanges


(403, 101204, 2508)

In [31]:
for i, e in enumerate(sp2.unlinked):
    if e['type'] == 'technosphere':
        print(e['name'], e['unit'], e['categories'])
    if i > 20000:
        break

Sugar beet pulp, dried, from pulp drying, at plant Economic/DE kilogram ('Materials/fuels',)
Maize gluten meal, consumption mix, at feed compound plant Economic/NL kilogram ('Materials/fuels',)
Wheat grain, conventional, national average, animal feed, at farm gate kilogram ('Materials/fuels',)
Municipal waste collection service by 21 metric ton lorry {GLO}| market for | Alloc Rec, U ton kilometer ('Electricity/heat',)
Sunflower seed meal, consumption mix, at feed compound plant Economic/NL kilogram ('Materials/fuels',)
Linseed, at farm Economic/DE kilogram ('Materials/fuels',)
Triticale, at farm Economic/DE kilogram ('Materials/fuels',)
Waste in incineration kilogram ('Final waste flows',)
Waste in inert landfill kilogram ('Final waste flows',)
Cathode loss kilogram ('Final waste flows',)
Dross kilogram ('Final waste flows',)
Dross for recycling kilogram ('Final waste flows',)
Aluminium waste kilogram ('Final waste flows',)
Cathode iron ingots waste kilogram ('Final waste flows',)
Mine

In [32]:
db.search('wastewater average market')

['market for wastewater, average' (cubic meter, Europe without Switzerland, None),
 'market for wastewater, average' (cubic meter, CH, None),
 'market for wastewater, average' (cubic meter, RoW, None),
 'primary lead production from concentrate' (kilogram, GLO, None),
 'primary lead production from concentrate' (kilogram, GLO, None),
 'primary lead production from concentrate' (kilogram, GLO, None)]

In [33]:
migration_data_wwav = {
    'fields': ['name'],
    'data': [
        (
            # First element is input data in the order of `fields` above
            ('Wastewater, average {GLO}| market for | Alloc Rec, U',),
            # Second element is new values 
            { 
                'name': 'market for wastewater, average',
                'location': 'Europe without Switzerland',
            }
        ),
        (
            # First element is input data in the order of `fields` above
            ('Municipal waste collection service by 21 metric ton lorry {GLO}| market for | Alloc Rec, U',),
            # Second element is new values 
            { 
                'name': 'market for municipal waste collection service by 21 metric ton lorry',
                'location': 'RoW',
            }
        ),
        (
            # First element is input data in the order of `fields` above
            ('Electricity, low voltage, at grid SP/DE',),
            # Second element is new values 
            { 
                'name': 'market for electricity, low voltage',
                'location': 'DE',
            }
        ),
        (
            # First element is input data in the order of `fields` above
            ('Light fuel oil, burned in boiler 100kW condensing, non-modulating SP/CH',),
            # Second element is new values 
            { 
                'name': 'heat production, light fuel oil, at boiler 100kW condensing, non-modulating',
                'location': 'Europe without Switzerland',
            }
        ),
        (
            # First element is input data in the order of `fields` above 
            ('market for transport, passenger car','meter',),
            # Second element is new values 
            {
                'name': 'market for transport, passenger car',
                'reference product': 'transport, passenger car',
                'unit': 'kilometer',
                'multiplier': 1e-3
                
            }
        ),
        (
            # First element is input data in the order of `fields` above
            ('Natural gas, burned in boiler condensing modulating <100kW SP/RER',),
            # Second element is new values 
            { 
                'name': 'heat production, natural gas, at boiler condensing modulating <100kW',
                'location': 'Europe without Switzerland',
            }
        ),
        (
            # First element is input data in the order of `fields` above
            ('Oxygen, liquid, at plant SP/RER',('Materials/fuels',)),
            # Second element is new values 
            { 
                'name': 'market for oxygen, liquid',
                'location': 'RER',
            }
        )
    ]
}

Migration('new-ecoinvent-wwav').write(
    migration_data_wwav, 
    description='New datasets in ecoinvent 3.3'
)

In [34]:
sp2.migrate('new-ecoinvent-wwav')
sp2.match_database('ecoinvent 3.3 cutoff',ignore_categories=True)
sp2.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
403 datasets
101204 exchanges
2488 unlinked exchanges
  Type biosphere: 192 unique unlinked exchanges
  Type technosphere: 19 unique unlinked exchanges


(403, 101204, 2488)

In [35]:
sp2.write_excel()

Wrote matching file to:
/home/oney/.local/share/Brightway3/Import-Quantis-OFP-version-4.ba90a3459a565ec10e14da8d23fa4b2d/output/db-matching-MeatDatasets.xlsx


#### link wheat agri

irgendwie wird so der harvester aus ag auch automatisch verlinkt

In [36]:
ag.search('wheat')

['Wheat grain, conventional, national average, animal feed, at farm gate' (kilogram, GB, None)]

In [37]:
migration_data_whAG = {
    'fields': ['name',],
    'data': [
        (
            # First element is input data in the order of `fields` above
            ('Wheat grain, conventional, national average, animal feed, at farm gate',),
            # Second element is new values 
            { 
                'name': 'Wheat grain, conventional, national average, animal feed, at farm gate',
            }
        )
    ]
}

Migration('new-ecoinvent-whAG').write(
    migration_data_whAG, 
    description='New datasets in ecoinvent 3.3'
)

In [38]:
sp2.migrate('new-ecoinvent-whAG')
sp2.match_database('AGs',ignore_categories=True)
sp2.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
403 datasets
101204 exchanges
2479 unlinked exchanges
  Type biosphere: 192 unique unlinked exchanges
  Type technosphere: 17 unique unlinked exchanges


(403, 101204, 2479)

In [39]:
for i, e in enumerate(sp2.unlinked):
    if e['type'] == 'technosphere':
        print(e['name'], e['unit'], e['categories'])
    if i > 20000:
        break

Sugar beet pulp, dried, from pulp drying, at plant Economic/DE kilogram ('Materials/fuels',)
Maize gluten meal, consumption mix, at feed compound plant Economic/NL kilogram ('Materials/fuels',)
Sunflower seed meal, consumption mix, at feed compound plant Economic/NL kilogram ('Materials/fuels',)
Linseed, at farm Economic/DE kilogram ('Materials/fuels',)
Triticale, at farm Economic/DE kilogram ('Materials/fuels',)
Waste in incineration kilogram ('Final waste flows',)
Waste in inert landfill kilogram ('Final waste flows',)
Cathode loss kilogram ('Final waste flows',)
Dross kilogram ('Final waste flows',)
Dross for recycling kilogram ('Final waste flows',)
Aluminium waste kilogram ('Final waste flows',)
Cathode iron ingots waste kilogram ('Final waste flows',)
Mineral waste, from mining kilogram ('Final waste flows',)
Waste in bioactive landfill kilogram ('Final waste flows',)
Dust, unspecified kilogram ('Final waste flows',)
Refined rapeseed oil, from crushing (pressing), at plant Econom

In [40]:
sp2.write_excel()

Wrote matching file to:
/home/oney/.local/share/Brightway3/Import-Quantis-OFP-version-4.ba90a3459a565ec10e14da8d23fa4b2d/output/db-matching-MeatDatasets.xlsx


## replace blonks data

#### Used by Raphael: refined rape oil 

datensatz ist doppelt.  Einer haben wir im file selber umbenannt zu rape oil mill operation4. 
Aufgrund des final GWP scheint es so zu sein, dass der ohne 4 der richtige ist.

In [41]:
db.search('rape oil mill operation')

['rape oil mill operation' (kilogram, Europe without Switzerland, None),
 'rape oil mill operation' (kilogram, RoW, None),
 'rape oil mill operation' (kilogram, CH, None),
 'rape oil mill operation' (kilogram, Europe without Switzerland, None),
 'rape oil mill operation' (kilogram, RoW, None),
 'rape oil mill operation' (kilogram, CH, None),
 'cottonseed oil mill operation' (kilogram, GLO, None),
 'cottonseed oil mill operation' (kilogram, GLO, None)]

In [42]:
migration_data_rap = {
    'fields': ['name','categories'],
    'data': [
        (
            # First element is input data in the order of `fields` above
            ('Refined rapeseed oil, from crushing (pressing), at plant Economic/DE',('Materials/fuels',)),
            # Second element is new values 
            { 
                'name': 'rape oil mill operation',
                'reference product':'rape oil, crude',
                'unit':'kilogram',
                'location': 'Europe without Switzerland',
                
            }
        )
    ]
}

Migration('new-ecoinvent_rap').write(
    migration_data_rap, 
    description='New datasets in ecoinvent 3.3'
)

In [43]:
sp2.migrate('new-ecoinvent_rap')
sp2.match_database('ecoinvent 3.3 cutoff',ignore_categories=True)
sp2.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
Couldn't apply strategy link_iterable_by_fields:
	Object in source database can't be uniquely linked to target database.
Problematic dataset is:
{'filename': '(missing)',
 'location': 'Europe without Switzerland',
 'name': 'rape oil mill operation',
 'unit': 'kilogram'}
Possible targets include (at least one not shown):
[{'filename': 'b5810020-570c-4d70-a208-c28f4048a4bb_3719fbf0-0ae5-4e1d-9668-e5f2cecf9a43.spold',
  'location': 'Europe without Switzerland',
  'name': 'rape oil mill operation',
  'unit': 'kilogram'}]
403 datasets
101204 exchanges
2479 unlinked exchanges
  Type biosphere: 192 unique unlinked exchanges
  Type technosphere: 17 unique unlinked exchanges


(403, 101204, 2479)

#### Used by Raphael:  Maize gluten meal, from wet milling

In [44]:
db.search('maize grain feed organic')

['maize grain, feed production, organic' (kilogram, CH, None),
 'market for maize grain, feed, organic' (kilogram, GLO, None),
 'maize grain, feed production, organic' (kilogram, RoW, None),
 'maize grain, feed production' (kilogram, RoW, None),
 'maize grain, feed production' (kilogram, CA-QC, None),
 'barley grain, feed production, organic' (kilogram, CH, None),
 'wheat grain, feed production, organic' (kilogram, CH, None),
 'rye grain, feed production, organic' (kilogram, CH, None)]

In [45]:
migration_data_ma = {
    'fields': ['name', 'categories'],
    'data': [
        (
            # First element is input data in the order of `fields` above
            ('Maize gluten meal, from wet milling (gluten drying), at plant Economic/DE',('Materials/fuels',)),
            # Second element is new values 
            { 
                'name': 'maize grain, feed production, organic',
                'reference product':'maize grain, feed, organic',
                'location': 'CH',
                'unit':'kilogram',
            }
        )
    ]
}

Migration('new-ecoinvent_ma').write(
    migration_data_ma, 
    description='New datasets in ecoinvent 3.3'
)

In [46]:
sp2.migrate('new-ecoinvent_ma')
sp2.match_database('ecoinvent 3.3 cutoff',ignore_categories=True)
sp2.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
Couldn't apply strategy link_iterable_by_fields:
	Object in source database can't be uniquely linked to target database.
Problematic dataset is:
{'filename': '(missing)',
 'location': 'Europe without Switzerland',
 'name': 'rape oil mill operation',
 'unit': 'kilogram'}
Possible targets include (at least one not shown):
[{'filename': 'b5810020-570c-4d70-a208-c28f4048a4bb_1ef8a081-f479-4c8e-bba3-aba6d37de904.spold',
  'location': 'Europe without Switzerland',
  'name': 'rape oil mill operation',
  'unit': 'kilogram'}]
403 datasets
101204 exchanges
2479 unlinked exchanges
  Type biosphere: 192 unique unlinked exchanges
  Type technosphere: 17 unique unlinked exchanges


(403, 101204, 2479)

#### Used by Deborah

check
- besser market oder production
- sunflower seed meal nicht in ecoinvent 

rest einigermassen ok
    

In [47]:
for i, e in enumerate(sp2.unlinked):
    if e['type'] == 'technosphere':
        print(e['name'], e['unit'], e['categories'])
    if i > 20000:
        break

Sugar beet pulp, dried, from pulp drying, at plant Economic/DE kilogram ('Materials/fuels',)
Maize gluten meal, consumption mix, at feed compound plant Economic/NL kilogram ('Materials/fuels',)
Sunflower seed meal, consumption mix, at feed compound plant Economic/NL kilogram ('Materials/fuels',)
Linseed, at farm Economic/DE kilogram ('Materials/fuels',)
Triticale, at farm Economic/DE kilogram ('Materials/fuels',)
Waste in incineration kilogram ('Final waste flows',)
Waste in inert landfill kilogram ('Final waste flows',)
Cathode loss kilogram ('Final waste flows',)
Dross kilogram ('Final waste flows',)
Dross for recycling kilogram ('Final waste flows',)
Aluminium waste kilogram ('Final waste flows',)
Cathode iron ingots waste kilogram ('Final waste flows',)
Mineral waste, from mining kilogram ('Final waste flows',)
Waste in bioactive landfill kilogram ('Final waste flows',)
Dust, unspecified kilogram ('Final waste flows',)
rape oil mill operation kilogram ('Materials/fuels',)
maize gra

In [48]:
db.search('sunflower')

['sunflower production' (kilogram, UA, None),
 'sunflower production' (kilogram, HU, None),
 'sunflower production' (kilogram, RU, None),
 'sunflower production' (kilogram, FR, None),
 'sunflower production' (kilogram, RoW, None),
 'sunflower production' (kilogram, ES, None),
 'market for sunflower silage' (kilogram, GLO, None),
 'market for sunflower seed' (kilogram, GLO, None),
 'catch crop growing, sunflower, August-October, organic fertiliser 30 kg N' (kilogram, CH, None),
 'catch crop growing, sunflower, August-October, organic fertiliser 20 kg N' (kilogram, RoW, None),
 'catch crop growing, sunflower, August-October, organic fertiliser 20 kg N' (kilogram, CH, None),
 'catch crop growing, sunflower, August-October, organic fertiliser 30 kg N' (kilogram, RoW, None),
 'sunflower seed production, for sowing' (kilogram, GLO, None),
 'market for sunflower seed, for sowing' (kilogram, GLO, None),
 'sunflower production, Swiss integrated production, intensive' (kilogram, CH, None),
 'mar

In [49]:
migration_data_li = {
    'fields': ['name', 'categories'],
    'data': [
        (
            # First element is input data in the order of `fields` above
            ('Linseed, at farm Economic/DE',('Materials/fuels',)),
            # Second element is new values 
            { 
                'name': 'market for linseed seed, at farm',
                'location': 'GLO',

            }
        ),
        (
            # First element is input data in the order of `fields` above
            ('Triticale, at farm Economic/DE',('Materials/fuels',)),
            # Second element is new values 
            { 
                'name': 'wheat grain, feed production, organic',
                'location': 'CH',

            }
        ),
        (
            # First element is input data in the order of `fields` above
            ('Sugar beet pulp, dried, from pulp drying, at plant Economic/DE',('Materials/fuels',)),
            # Second element is new values 
            { 
                'name': 'market for sugar beet pulp',
                'location': 'GLO',

            }
        ),
        (
            # First element is input data in the order of `fields` above
            ('Maize gluten meal, consumption mix, at feed compound plant Economic/NL',('Materials/fuels',)),
            # Second element is new values 
            { 
                'name': 'maize grain, feed production, organic',
                'location': 'CH',

            }
        ),
        (
            # First element is input data in the order of `fields` above
            ('Sunflower seed meal, consumption mix, at feed compound plant Economic/NL',('Materials/fuels',)),
            # Second element is new values 
            { 
                'name': 'sunflower production',
                'location': 'FR',

            }
        )
    ]
}

Migration('new-ecoinvent_li').write(
    migration_data_li, 
    description='New datasets in ecoinvent 3.3'
)

In [50]:
sp2.migrate('new-ecoinvent_li')
sp2.match_database('ecoinvent 3.3 cutoff',ignore_categories=True)
sp2.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
Couldn't apply strategy link_iterable_by_fields:
	Object in source database can't be uniquely linked to target database.
Problematic dataset is:
{'filename': '(missing)',
 'location': 'Europe without Switzerland',
 'name': 'rape oil mill operation',
 'unit': 'kilogram'}
Possible targets include (at least one not shown):
[{'filename': 'b5810020-570c-4d70-a208-c28f4048a4bb_3719fbf0-0ae5-4e1d-9668-e5f2cecf9a43.spold',
  'location': 'Europe without Switzerland',
  'name': 'rape oil mill operation',
  'unit': 'kilogram'}]
403 datasets
101204 exchanges
2461 unlinked exchanges
  Type biosphere: 192 unique unlinked exchanges
  Type technosphere: 12 unique unlinked exchanges


(403, 101204, 2461)

## fix most important biosphere flows

In [51]:
for i, e in enumerate(sp2.unlinked):
    if e['type'] == 'biosphere':
        print(e['name'], e['unit'], e['categories'])
    if i > 20000:
        break

Butyric acid, 4-(2,4-dichlorophenoxy)- kilogram ('soil', 'agricultural')
Copper, Cu 0.2%, in mixed ore kilogram ('natural resource', 'in ground')
Gold, Au 1.8E-4%, in mixed ore kilogram ('natural resource', 'in ground')
Gold, Au 5.4E-4%, Ag 1.5E-5%, in ore kilogram ('natural resource', 'in ground')
Gold, Au 6.8E-4%, Ag 1.5E-4%, in ore kilogram ('natural resource', 'in ground')
Gold, Au 9.7E-5%, Ag 7.6E-5%, in ore kilogram ('natural resource', 'in ground')
Lead, Pb 3.6E-1%, in mixed ore kilogram ('natural resource', 'in ground')
Silver, Ag 1.5E-4%, Au 6.8E-4%, in ore kilogram ('natural resource', 'in ground')
Silver, Ag 1.5E-5%, Au 5.4E-4%, in ore kilogram ('natural resource', 'in ground')
Silver, Ag 5.4E-3%, in mixed ore kilogram ('natural resource', 'in ground')
Silver, Ag 7.6E-5%, Au 9.7E-5%, in ore kilogram ('natural resource', 'in ground')
Strontium kilogram ('natural resource', 'in ground')
Transformation, from unspecified, natural square meter ('natural resource', 'land')
Transfo

In [52]:
migration_data_bioM = {
    'fields': ['name','categories','type','unit'],
    'data': [
        (
            # Achtung: nicht sicher ob mit fossil oder non-fossil ersetzen->check CF!!
            ('Carbon dioxide',('air',), 'biosphere', 'kilogram'),
            
            { 
                'name': 'Carbon dioxide, fossil',               
            }
        ),
        (
            # Achtung: nicht sicher ob mit fossil oder non-fossil ersetzen->check CF!!
            ('Carbon monoxide',('air',), 'biosphere', 'kilogram'),
            
            { 
                'name': 'Carbon monoxide, fossil',               
            }
        ),   
        (
            # First element is input data in the order of `fields` above
            ('Methane',('air',), 'biosphere', 'kilogram'),
            # Second element is new values 
            { 
                'name': 'Methane, fossil',              
            }
        ),
        (
            # First element is input data in the order of `fields` above
            ('Methane',('air','non-urban air or from high stacks',), 'biosphere', 'kilogram'),
            # Second element is new values 
            { 
                'name': 'Methane, fossil',              
            }
        ),
        (
            
            ('Nitrogen monoxide',('air', 'non-urban air or from high stacks'), 'biosphere', 'kilogram'),
            
            { 
                'name': 'Nitrogen oxides',              
            }
        ),
        (
            
            ('Nitrogen monoxide',('air'), 'biosphere', 'kilogram'),
            
            { 
                'name': 'Nitrogen oxides',              
            }
        ) 
    ]
}

Migration('new-ecoinvent-bioM').write(
    migration_data_bioM, 
    description='New datasets in ecoinvent 3.3'
)

In [53]:
sp2.migrate('new-ecoinvent-bioM')
sp2.apply_strategies()
sp2.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: normalize_units
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
Applying strategy: link_iterable_by_fields
Applied 19 strategies in 6.82 seconds
403 d

(403, 101204, 2404)

In [54]:
bp.search('nitrogen')

['Nitrogen' (kilogram, None, ('soil', 'agricultural')),
 'Nitrogen' (kilogram, None, ('soil', 'industrial')),
 'Nitrogen' (kilogram, None, ('soil',)),
 'Nitrogen' (kilogram, None, ('water', 'ocean')),
 'Nitrogen' (kilogram, None, ('water', 'surface water')),
 'Nitrogen' (kilogram, None, ('water', 'ground-, long-term')),
 'Nitrogen' (kilogram, None, ('air',)),
 'Nitrogen' (kilogram, None, ('natural resource', 'in air')),
 'Nitrogen' (kilogram, None, ('soil', 'forestry')),
 'Nitrogen' (kilogram, None, ('water', 'ground-')),
 'Nitrogen' (kilogram, None, ('water',)),
 'Nitrogen fluoride' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Nitrogen oxides' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Nitrogen fluoride' (kilogram, None, ('air', 'low population density, long-term')),
 'Nitrogen fluoride' (kilogram, None, ('air', 'urban air close to ground')),
 'Nitrogen oxides' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Nitro

## drop unlinked exchanges

betrifft momentan alle waste flows

In [55]:
sp2.drop_unlinked(i_am_reckless=True)

Applying strategy: drop_unlinked
Applied 1 strategies in 0.06 seconds


In [56]:
sp2.match_database('ecoinvent 3.3 cutoff', ignore_categories=True)

Applying strategy: link_iterable_by_fields


In [57]:
sp2.statistics()

403 datasets
98800 exchanges
0 unlinked exchanges
  


(403, 98800, 0)

## write database

In [58]:
quantisdata = sp2.write_database()

Title: Writing activities to SQLite3 database:
  Started: 03/16/2017 16:11:02
  Finished: 03/16/2017 16:11:29
  Total time elapsed: 00:00:27
  CPU %: 83.10
  Memory %: 14.62
Created database: MeatDatasets
Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:27


In [59]:
quantisdata

Brightway2 SQLiteBackend: MeatDatasets

In [10]:
list(databases)

['AGs', 'ecoinvent 3.3 cutoff', 'MeatDatasets', 'biosphere3']

# calculate LCIA

try first sth from ecoinvent

In [11]:
act = Database("MeatDatasets").search('suckler average')[1]
act


'Suckler beef average, conventional (Quantis, from Agribalyse) UP/DE' (kilogram, None, None)

In [12]:
lca = LCA(
    {act.key: 1}, 
    method=('IPCC 2013', 'climate change', 'GWP 100a'),
)
lca.lci()
lca.lcia()
lca.score

18.4052599249694

## Contribution Analysis

In [13]:
from bw2analyzer import ContributionAnalysis

In [16]:
help(ContributionAnalysis().annotated_top_processes)

Help on method annotated_top_processes in module bw2analyzer.contribution:

annotated_top_processes(lca, names=True, **kwargs) method of bw2analyzer.contribution.ContributionAnalysis instance
    Get list of most damaging processes in an LCA, sorted by ``abs(direct impact)``.
    
    Returns a list of tuples: ``(lca score, supply, activity)``. If ``names`` is False, they returns the process key as the last element.



In [17]:
ContributionAnalysis().annotated_top_processes(lca, names=False)

[(3.2981247535411664,
  1.1423420410952435e-05,
  ('MeatDatasets', '5eedd964fd3c69e7af816ff967d3c8b3')),
 (2.6288027557898612,
  7.2811412651059102e-06,
  ('MeatDatasets', '1dd7aa59e551b9b3036f06e767b28a25')),
 (2.2530833682989759,
  1.4631230486744122e-05,
  ('MeatDatasets', '490f41153687f5bf1aa979c33f0285e4')),
 (1.3520051882298609,
  1.2114514515226485e-05,
  ('MeatDatasets', '0690e319eb05a54860db1bbf31a1c124')),
 (1.0494427408896485,
  22.821963651731178,
  ('MeatDatasets', 'cc01ff7bf1c7514cd047c42c80525b18')),
 (1.0422489442192411,
  2.2341282841442851e-05,
  ('MeatDatasets', '67d3ab6b3b3e8392cd85ac0db53d5a79')),
 (0.88585165714017911,
  4.3020639668352008,
  ('ecoinvent 3.3 cutoff', '83b638224d563673a9da377aaeb852d9')),
 (0.59358624441363539,
  19.197928421684416,
  ('MeatDatasets', 'bc86ef44d14db0476d96157e9a479024')),
 (0.59193865510611443,
  5.126804769108502e-06,
  ('MeatDatasets', 'a4fd663f1ce4809be0b31465cdf16e72')),
 (0.54330172337387106,
  8.7488132218519834e-06,
  ('Meat

In [18]:
help(ContributionAnalysis().annotated_top_emissions)

Help on method annotated_top_emissions in module bw2analyzer.contribution:

annotated_top_emissions(lca, names=True, **kwargs) method of bw2analyzer.contribution.ContributionAnalysis instance
    Get list of most damaging biosphere flows in an LCA, sorted by ``abs(direct impact)``.
    
    Returns a list of tuples: ``(lca score, inventory amount, activity)``. If ``names`` is False, they returns the process key as the last element.



In [81]:
ContributionAnalysis().annotated_top_emissions(lca)

[(11.653903594071792,
  0.4089088980376071,
  'Methane, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (3.3550709038030657,
  0.012670207924730675,
  'Dinitrogen monoxide' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (1.5937783154278484,
  1.5937783154278475,
  'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (0.80535814152737162,
  0.80535814152737206,
  'Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground'))),
 (0.38303347947197885,
  0.38303347947197852,
  'Carbon dioxide, fossil' (kilogram, None, ('air',))),
 (0.32941719902933975,
  0.0012440227122929152,
  'Dinitrogen monoxide' (kilogram, None, ('air', 'urban air close to ground'))),
 (0.14777017123808653,
  0.00497542651320328,
  'Methane, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks'))),
 (0.065395837196283488,
  0.065395837196283474,
  'Carbon dioxide, from soil or biomass stock' (kilogram

## Create report

In [5]:
from bw2analyzer.report import SerializedLCAReport
test = SerializedLCAReport(
    activity={act.key: 1},
    method=('IPCC 2013', 'climate change', 'GWP 100a'),
    iterations=10,
)

NameError: name 'act' is not defined

In [85]:
test.calculate()

FD
CA
hinton
treemap
herfindahl
concentration
MC:
Entered get_monte_carlo


DatabaseError: database disk image is malformed

In [0]:
test.report['monte carlo']['histogram']= list(test.report['monte carlo']['histogram'])

In [0]:
test.report['monte carlo']['smoothed'] = list(test.report['monte carlo']['smoothed'])

In [0]:
test.write()

In [0]:
test.report

In [0]:
sorted(str(get_activity(key)) for key, _ in Method(('IPCC 2013', 'climate change', 'GWP 100a')).load())

See all flows with their characterization factors

In [4]:
from brightway2 import *
# method = methods.random()
methods = Method(('IPCC 2013', 'climate change', 'GWP 100a')).load()
for key, cf in methods:
    print(get_activity(key), cf)

UnknownObject: This object is not registered and has no data